In [20]:
import meteva.base as meb
import numpy as np
import datetime 
import pandas as pd
from netCDF4 import Dataset
import xarray as xr
from wrf import getvar, ALL_TIMES
import wrf
import copy
import glob
import meteva.method as mem
import meteva.base as meb
import meteva.product as mpd
import meteva.perspact as mps # 透视分析模块
import meteva
import salem
from copy import deepcopy
from time import sleep
from datetime import timedelta
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
import sys
sys.path.append("/mnt/d/work/sunhh/script/function/")
from wrfout_need import *
from pathlib import Path

######## 设置
lead_list=["03","06"]
sta_dir = glob.glob('/mnt/d/work/obs/CHN_sta/山东/山东20220707-*.csv' ) #所需的观测数据csv
data_path="/mnt/d/work/sunhh/data/run_cyc/" #wrfout.nc存放的位置
wrfout_path="/mnt/d/work/sunhh/data/run3/ctl2/wrfout.nc"  #wrfout网格信息用于插值
Province="山东省"  #目标省份
station_file = "/mnt/d/work/sunhh/data/station/SURF_CHN_LIST.xlsx"  #观测站信息文件位置

####读取指定省份的站点信息
# station_file = "/mnt/d/work/sunhh/data/station/SURF_CHN_LIST.xlsx"
station=pd.read_excel(station_file)
station=station[station.Province==Province]
station_info = pd.DataFrame(list(zip(station.Station_Id_C,station.Lat,station.Lon,station.Alti)), columns=["id","lat","lon","Alti"])

#读取wrfout数据信息，用于插值
# wrfout_path="/mnt/d/work/sunhh/data/run3/ctl2/wrfout.nc"
wrfout_salem = salem.open_wrf_dataset(wrfout_path)
wrf_sta_idx=wrf2sta_id(station_info.id,station_info.lat,station_info.lon,wrfout_salem)
wrf_sta_idx=pd.merge(wrf_sta_idx,station_info[["id","Alti"]],on=["id"])

In [21]:
#读取所需的csv数据
# sta_dir = glob.glob('/mnt/d/work/obs/CHN_sta/山东/山东20220707-*.csv' )
usecols = ['Station_Id_C', 'Year', 'Mon', 'Day', 'Hour', 'PRS','TEM', 'PRE_1h','RHU','WIN_D_Avg_2mi','WIN_S_Avg_2mi'] 
df = pd.DataFrame() 
na_values = [999017.0000,999998.0000,999999.0000]  # 缺测，读取时替换为Nan
for sta_csv in sta_dir:
    df_t = pd.read_csv(sta_csv,parse_dates=False, na_values=na_values,
                    engine='python', usecols=usecols, encoding='utf-8')
    df=pd.concat([df,df_t])
   
#去除过大的异常值
df.where(df.iloc[:,5::]<10000,None,inplace=True)
df.where(df["WIN_S_Avg_2mi"]<100,None,inplace=True)
df.rename(columns={"Station_Id_C":"id"},inplace=True)

data = pd.merge(df[['id','Year','Mon','Day','Hour','PRE_1h','TEM','PRS','RHU','WIN_D_Avg_2mi','WIN_S_Avg_2mi']],
                station_info[['id','lat','lon','Alti']],on='id')
data=PreProcess(data)

data.insert(loc=0,column="level",value=0)
data.insert(loc=2,column="dtime",value=0)
data.rename(columns={ "WIN_D_Avg_2mi":"angle","WIN_S_Avg_2mi":"speed","PRE_1h":"pre_1h_obs",
                     "TEM":"t_obs","PRS":"p_obs","RHU":"rh_obs","Alti":"Alti_obs"},inplace=True)
datalist=["level","time","dtime","id","lon","lat","pre_1h_obs","t_obs",'p_obs','rh_obs','Alti_obs']
data_uv=data[["level","time","dtime","id","lon","lat","speed","angle"]]
data_uv=meb.speed_angle_to_wind(data_uv)
data_obs=pd.merge(data_uv[["time","id","u","v"]],data[datalist],on=["time","id"])
data_obs.rename(columns={"u":"u_obs","v":"v_obs"},inplace=True)
data_obs=data_obs[["level","time","dtime","id","lon","lat","pre_1h_obs","t_obs",
               'p_obs','rh_obs','u_obs','v_obs','Alti_obs']]
data_obs=data_obs.drop_duplicates(subset=None, keep='first', inplace=False)
data_obs

,level,time,dtime,id,lon,lat,pre_1h_obs,t_obs,p_obs,rh_obs,u_obs,v_obs,Alti_obs
0,0,2022-07-07 00:00:00,0,54709,116.0667,37.2333,0.0,25.8,1000.6,82,0.739247,1.750290,24.8
1,0,2022-07-07 01:00:00,0,54709,116.0667,37.2333,0.0,28.0,1000.6,74,0.051029,-0.698138,24.8
2,0,2022-07-07 02:00:00,0,54709,116.0667,37.2333,0.0,29.8,1000.5,72,1.679722,-0.261793,24.8
3,0,2022-07-07 03:00:00,0,54709,116.0667,37.2333,0.0,31.3,1000.3,66,-0.739981,0.944684,24.8
4,0,2022-07-07 04:00:00,0,54709,116.0667,37.2333,0.0,32.3,1000.0,57,0.649704,-0.260547,24.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
990880,0,2022-08-02 19:00:00,0,58032,118.6500,34.9500,0.0,27.4,997.4,93,-1.707211,1.826316,90.1
990881,0,2022-08-02 20:00:00,0,58032,118.6500,34.9500,0.0,27.2,997.4,94,-1.512695,1.863265,90.1
990882,0,2022-08-02 21:00:00,0,58032,118.6500,34.9500,0.0,27.1,997.8,92,-1.628287,1.624402,90.1
990883,0,2022-08-02 22:00:00,0,58032,118.6500,34.9500,0.0,27.4,998.1,89,-1.742097,1.930051,90.1


In [22]:
#读取指定wrfout.nc的信息，插值到站点上
sta_area = data_obs[data_obs.time==data_obs.time[0]][["id","lon","lat"]]
wrfout_tmp = salem.open_wrf_dataset(wrfout_path)
wrf_sta_idx = wrf2sta_id(sta_area.id,sta_area.lat,sta_area.lon,wrfout_tmp)
wrf_sta_idx=pd.merge(wrf_sta_idx,station_info[["id","Alti"]],on=["id"])
wrf_sta_idx

,id,lat,lon,wrf_lat_idx,wrf_lon_idx,Alti
0,54709,37.2333,116.0667,57,42,24.8
1,54712,37.2000,116.8167,57,47,18.5
2,54714,37.3500,116.3333,58,43,25.4
3,54715,37.3167,116.5167,58,45,17.6
4,54716,37.6667,116.8000,61,47,17.3
...,...,...,...,...,...,...
104,58022,34.7667,117.4833,35,53,83.4
105,58024,34.8667,117.5833,36,54,77.7
106,58025,34.6000,117.7167,34,55,29.3
107,58030,34.8833,118.0167,37,57,44.5


In [23]:
#把所需的wrfout处理成csv
list_all=[]
for lead in lead_list:
    list1=sorted(glob.glob(data_path+f"/*lead{lead}.nc"))
    list2=[]
    for i in list1:
        wrfout_path = i
        print(wrfout_path)
        list2.append(wrf2sta_2(wrf_sta_idx,wrfout_path))
    list_all.append(list2)


/mnt/d/work/sunhh/data/run_cyc/wrfout_cv3_lead03.nc
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 /mnt/d/work/sunhh/data/run_cyc/wrfout_cv5_be3_lead03.nc
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 /mnt/d/work/sunhh/data/run_cyc/wrfout_cv5_be5_lead03.nc
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 

In [35]:
#输出wrfout插值到站点的csv文件
for i,lead in enumerate(lead_list):
    list1=sorted(glob.glob(data_path+f"/*lead{lead}.nc"))
    dtime=int(lead)
    data_obs_tmp=deepcopy(data_obs)
    data_lead_tmp=list_all[i]
    for j in range(0,len(list_all[i])):
        data_member=data_lead_tmp[j]
        data_merge_list=["level","time","dtime","id","lon","lat"]
        exec(f"da{j}=pd.merge(data_obs[data_merge_list],data_member,on=['time','id'],how='inner')")
        exec(f"da{j}.time = da{j}.time -timedelta(hours=dtime)")
        exec(f"da{j}.dtime = dtime")
        exec(f"da{j}.to_csv(data_path+Path(list1[j]).name.split('.')[0]+'.csv',encoding='utf-8')")